In [7]:
# this is unrelated to the class .. It just helps displaying all outputs in a cell instead of just last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Data Cleaning

your data is not always nice and clean. There are messing fields. There are rows that are comments and does not belong to the data. How to handle that while reading and after reading and how to get rid of the bad values and fix it is what we will be doing today

If you look at the file ex4.csv, you will find that rows 0,2,3 do not belong to the data, they are just comments added by the analyst. We need to skip them

In [3]:
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [4]:
import pandas as pd
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


For ex5.csv, if you load it, there will be some weird data point marked as NA. Pandas automatically understands that this is an invalid data point and marks it as NaN. Similarly, when there is a data point such as row 1, col c which is missing. Pandas also substitutes this as NaN. You can check if a frame has a NaN value using the function `isnull()`

In [9]:
!cat examples/ex5.csv
result = pd.read_csv('examples/ex5.csv')
result
pd.isnull(result)

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [26]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Filtering Out Missing Data

Multiple ways to get rid of Nans

In [12]:
result.dropna()

,something,a,b,c,d,message
2,three,9,10,11.0,12,foo


Try to create your own NaNs and clean them

In [13]:
from numpy import nan as NA
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data
cleaned

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


,0,1,2
0,1.0,6.5,3.0


In [14]:
data.dropna(how='all') # this will only delete if all rows is NaNs

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [15]:
data[4] = NA
data
data.dropna(axis=1, how='all') # this will delete columns full of NaNs

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [22]:
import numpy as np
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df
df.dropna()
df.dropna(thresh=2) # there has to be at least two Nans to drop

,0,1,2
0,-2.284006,NaN,NaN
1,-0.458565,NaN,NaN
2,0.563584,NaN,-0.789565
3,1.459659,NaN,-0.834648
4,1.024832,-1.132852,-0.221691
5,1.437201,0.015595,0.434185
6,0.983565,1.340429,1.391513


,0,1,2
4,1.024832,-1.132852,-0.221691
5,1.437201,0.015595,0.434185
6,0.983565,1.340429,1.391513


,0,1,2
2,0.563584,NaN,-0.789565
3,1.459659,NaN,-0.834648
4,1.024832,-1.132852,-0.221691
5,1.437201,0.015595,0.434185
6,0.983565,1.340429,1.391513


### Filling In Missing Data

You do not have to always drop NA, you can also replace it with other values

In [18]:
df
df.fillna(0)

,0,1,2
0,0.251589,NaN,NaN
1,-0.272387,NaN,NaN
2,-0.500758,NaN,1.876878
3,0.370443,NaN,-0.587637
4,-1.952866,-0.086923,-0.459238
5,0.131840,0.898852,-0.902941
6,-0.895734,-0.200162,-1.418627


,0,1,2
0,0.251589,0.000000,0.000000
1,-0.272387,0.000000,0.000000
2,-0.500758,0.000000,1.876878
3,0.370443,0.000000,-0.587637
4,-1.952866,-0.086923,-0.459238
5,0.131840,0.898852,-0.902941
6,-0.895734,-0.200162,-1.418627


You can also replace with different value for each row

In [19]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,0.251589,0.500000,0.000000
1,-0.272387,0.500000,0.000000
2,-0.500758,0.500000,1.876878
3,0.370443,0.500000,-0.587637
4,-1.952866,-0.086923,-0.459238
5,0.131840,0.898852,-0.902941
6,-0.895734,-0.200162,-1.418627


And by now you know how `inplace` works

In [24]:
df.fillna(0, inplace=True)
df


,0,1,2
0,-2.284006,0.000000,0.000000
1,-0.458565,0.000000,0.000000
2,0.563584,0.000000,-0.789565
3,1.459659,0.000000,-0.834648
4,1.024832,-1.132852,-0.221691
5,1.437201,0.015595,0.434185
6,0.983565,1.340429,1.391513


,0,1,2
0,-2.284006,0.000000,0.000000
1,-0.458565,0.000000,0.000000
2,0.563584,0.000000,-0.789565
3,1.459659,0.000000,-0.834648
4,1.024832,-1.132852,-0.221691
5,1.437201,0.015595,0.434185
6,0.983565,1.340429,1.391513


You can also fill based on previous values

In [25]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df
df.fillna(method='ffill')
df.fillna(method='ffill', limit=2) # just fill the following two

,0,1,2
0,0.672522,0.645922,0.221794
1,0.475434,0.523883,-0.548328
2,-1.293808,NaN,-1.443121
3,2.348162,NaN,-0.072683
4,1.747174,NaN,NaN
5,0.517198,NaN,NaN


,0,1,2
0,0.672522,0.645922,0.221794
1,0.475434,0.523883,-0.548328
2,-1.293808,0.523883,-1.443121
3,2.348162,0.523883,-0.072683
4,1.747174,0.523883,-0.072683
5,0.517198,0.523883,-0.072683


,0,1,2
0,0.672522,0.645922,0.221794
1,0.475434,0.523883,-0.548328
2,-1.293808,0.523883,-1.443121
3,2.348162,0.523883,-0.072683
4,1.747174,NaN,-0.072683
5,0.517198,NaN,-0.072683


## Data Transformation

### Removing Duplicates

In [28]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [29]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [30]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [32]:
data['v1'] = range(7)
data
data.drop_duplicates(['k1']) # remove the duplicates from K1

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


,k1,k2,v1
0,one,1,0
1,two,1,1


You can also keep the last instead of removing it

In [33]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

Sometimes you wanna map the data to something else that they represent. Such as meat type to the animal it came from.

In [35]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [37]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [38]:
lowercased = data['food'].str.lower()
lowercased
data['animal'] = lowercased.map(meat_to_animal)
data

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


### Replacing Values

In [39]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [40]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [43]:
data.replace([-999, -1000], [np.nan, 0])
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Discretization and Binning

Binning is one of the skills that is important for a data scientist. It can reduce the data size and show plenty of insights and histograms

In [44]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [45]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [46]:
cats.codes
cats.categories
pd.value_counts(cats)

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [48]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

You can also cut the data into quartiles. i.e., equivalent quarters

In [50]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats
pd.value_counts(cats)

[(-3.0949999999999998, -0.701], (-0.0304, 0.644], (-3.0949999999999998, -0.701], (-3.0949999999999998, -0.701], (-3.0949999999999998, -0.701], ..., (-0.701, -0.0304], (0.644, 2.795], (0.644, 2.795], (-3.0949999999999998, -0.701], (0.644, 2.795]]
Length: 1000
Categories (4, interval[float64]): [(-3.0949999999999998, -0.701] < (-0.701, -0.0304] < (-0.0304, 0.644] < (0.644, 2.795]]

(0.644, 2.795]                   250
(-0.0304, 0.644]                 250
(-0.701, -0.0304]                250
(-3.0949999999999998, -0.701]    250
dtype: int64

In [8]:
!cat examples/csv_mindex.csv
parsed = pd.read_csv('examples/csv_mindex.csv',
                     index_col=['key1', 'key2'])
parsed

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [27]:
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


### Reading Text Files in Pieces

In [12]:
pd.options.display.max_rows = 10

In [13]:
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [14]:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [15]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

In [17]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

tot = pd.Series([])
tot
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
tot

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
     ...  
5    157.0
2    152.0
0    151.0
9    150.0
1    146.0
Length: 36, dtype: float64

In [24]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

In [26]:
result['key'].value_counts()

E    368
X    364
L    346
O    343
Q    340
    ... 
5    157
2    152
0    151
9    150
1    146
Name: key, Length: 36, dtype: int64